<a href="https://colab.research.google.com/github/velmer/retrieval-information/blob/master/lab-06/Modelo_Vetorial_ri_lab_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>